In [2]:
import json
from tqdm import tqdm
import pandas as pd

In [3]:
class SciRexDataPrepDoc():
    def __init__(self):
        self.v = 0
    
    def convert_to_df(self, words, sentences, anns):
        df = []
        kept = 0
        
        annotations = ['O']*len(words)
        for ann in anns:
            beg, end = int(ann[0]), int(ann[1])
            kept += 1
            annotations[beg] = 'B'
            
            for i in range(beg+1, end):
                annotations[i] = 'I'
        
        for sent in sentences:
            beg, end = sent[0], sent[1]
            cur_sent = {}
            cur_sent['word_labels'] = ','.join(annotations[beg:end])
            cur_sent['sentence'] = ' '.join(words[beg:end])
            df.append(cur_sent)
            
        df = pd.DataFrame(df)
        return df, kept            

In [4]:
class SciRexDataPrepDir():
    def __init__(self):
        self.v = 0

    def get_df(self, inpath, outpath = ''):
        df = pd.DataFrame(columns=['sentence', 'word_labels'])
        doc_prepper = SciRexDataPrepDoc()
        with open(inpath, 'r') as json_file:
            json_list = list(json_file)

        kept = 0
        #print(len(json_list))
        for json_str in tqdm(json_list):
            result = json.loads(json_str)
            words = result['words']
            anns = result['ner']
            sentences = result['sentences']
            cur_df, cur_kept = doc_prepper.convert_to_df(words, sentences, anns)
            kept += cur_kept
            df = pd.concat([df, cur_df], ignore_index = True)
        df.to_csv(outpath, header=True)
        print("total annotations: ", kept)
        return df
            

In [5]:
filepath = '../corpus/scirex/dev.jsonl'
outpath = '../corpus/scirex/scirex_train_bio.csv'
prepper = SciRexDataPrepDir()
df = prepper.get_df(filepath, outpath)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 66/66 [00:00<00:00, 673.98it/s]

total annotations:  23365


In [65]:
df

,sentence,word_labels
0,document : Attention Boosted Sequential Infere...,"O,O,B,I,I,I,I"
1,Attention mechanism has been proven effective ...,"B,I,O,O,O,O,O,B,I,I,O"
2,This paper proposes an attention boosted natur...,"O,O,O,O,B,I,I,I,I,I,O,B,O,O,B,I,O,B,I,I,I,I,I,..."
3,This makes the inference model aESIM has the a...,"O,O,O,B,I,I,O,O,O,O,O,O,O,O,O,O,O,O,O,B,I,I,O,..."
4,"The empirical studies on the SNLI , MultiNLI a...","O,O,O,O,O,B,O,B,O,B,O,O,O,B,O,O,O,O,O,B,I,O"
...,...,...
17866,We have presented a scene text detector that d...,"O,O,O,O,B,I,I,O,O,O,B,I,I,I,I,O,O,O,O,O,O,B,I,O"
17867,"By incorporating proper loss functions , the d...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
17868,The experiments on standard benchmarks confirm...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B,O,B,O"
17869,Possible directions for future research includ...,"O,O,O,O,O,O,O,O,O,O,O,O,B,I,O,O,B,I,I,I,I,O,O,..."
